In [1]:
from general_tools.notebook.gpu_utils import setup_one_gpu
GPU = 0
setup_one_gpu(GPU)

Picking GPU 0


In [2]:
from general_tools.notebook.tf import reset_tf_graph
from scipy.io import loadmat, savemat
from geo_tool import Point_Cloud
import os.path as osp
import matplotlib.pylab as plt
from scipy.spatial.distance import pdist
import numpy as np
import tensorflow as tf
import hdf5storage

In [3]:
from tflearn.layers.core import fully_connected
from tf_lab.point_clouds.encoders_decoders import encoder_with_convs_and_symmetry_new, decoder_with_fc_only
from tf_lab.data_sets.numpy_dataset import NumpyDataset
from tflearn.layers.conv import conv_2d
from sklearn.model_selection import train_test_split

In [4]:
%matplotlib inline

In [5]:
top_data_dir = '/orions4-zfs/projects/optas/DATA/OUT/latent_diff_maps/experiments/human_poses/'

In [6]:
n_classes = 8
members_per_class = 150
val_per = 0.05
test_per = 0.15
train_per = 1.0 - (val_per + test_per)
seed = 42

In [7]:
# LOAD input differences
in_diffs = osp.join(top_data_dir, '%d_60_30_consistent_diffs.mat' % members_per_class)
in_diffs = hdf5storage.loadmat(in_diffs)
n_shapes = len(in_diffs['ucb'])
diff_dims = in_diffs['ucb'][1][0].shape
temp = np.zeros(shape=(n_shapes, ) + diff_dims )
for i in xrange(n_shapes):
    temp[i] = in_diffs['ucb'][i][0]
in_diffs = temp
in_diffs[in_diffs < 10e-6] = 0

In [8]:
class_labels = np.zeros(n_shapes)
c = 0
for i in range(n_shapes):
    if i % members_per_class == 0:
        c += 1
    class_labels[i] = c
class_labels -= 1

In [9]:
all_ids = np.arange(n_shapes)
train_ids, rest_ids = train_test_split(all_ids, stratify=class_labels, train_size=train_per, random_state=seed)
test_ids, val_ids = train_test_split(rest_ids, stratify=class_labels[rest_ids], train_size=int(n_shapes*test_per), random_state=seed)
# np.sum(np.logical_and(test_ids>=300,  test_ids<400))
in_data = dict()
in_data['train'] = train_ids
in_data['test'] = test_ids
in_data['val'] = val_ids

/afs/cs.stanford.edu/u/optas/.local/lib/python2.7/site-packages/sklearn/model_selection/_split.py:2010: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [10]:
for s in ['train', 'test', 'val']:
    idx = in_data[s].copy()
    in_data[s] = NumpyDataset([in_diffs[idx], class_labels[idx], idx], ['diff_maps', 'labels', 'ids'])

['diff_maps', 'labels', 'ids']
['diff_maps', 'labels', 'ids']
['diff_maps', 'labels', 'ids']


In [11]:
reset_tf_graph()

In [12]:
with tf.variable_scope('diff_based_net') as scope:
    diffs_pl = tf.placeholder(tf.float32, shape = (None, ) + diff_dims)
    labels_pl = tf.placeholder(tf.int64, shape=[None])
    layer = fully_connected(diffs_pl, 64, activation='relu', weights_init='xavier')
    layer = fully_connected(layer, 64, activation='relu', weights_init='xavier')
    logits = fully_connected(layer, n_classes, activation='linear', weights_init='xavier')

In [13]:
prediction = tf.argmax(logits, axis=1)
target_ = labels_pl
correct_pred = tf.equal(prediction, target_)
avg_accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
one_hot_labels = tf.one_hot(labels_pl, depth=n_classes)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=one_hot_labels)
loss = tf.reduce_mean(cross_entropy)

In [14]:
learning_rate = 0.001
opt = tf.train.AdamOptimizer(learning_rate)
opt_step = opt.minimize(loss)

In [15]:
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
init=tf.global_variables_initializer()
sess.run(init)

In [16]:
n_epochs = 100
batch_size = 25

train_losses = []
val_losses = []
test_losses = []

train_data = in_data['train']
val_data = in_data['val']
test_data = in_data['test']

batches_for_epoch = train_data.n_examples / batch_size

In [17]:
for epoch in range(n_epochs):    
    for _ in range(batches_for_epoch):        
        batch_d, batch_l, _ = train_data.next_batch(batch_size)
        feed_dict = {diffs_pl:batch_d, labels_pl:batch_l}
        sess.run([opt_step, loss, avg_accuracy], feed_dict=feed_dict)
    
    feed_dict = {diffs_pl:train_data.diff_maps, labels_pl:train_data.labels}
    train_losses.append(sess.run([avg_accuracy], feed_dict=feed_dict))
    
    feed_dict = {diffs_pl:val_data.diff_maps, labels_pl:val_data.labels}
    val_losses.append(sess.run([avg_accuracy], feed_dict=feed_dict))
        
    feed_dict = {diffs_pl:test_data.diff_maps, labels_pl:test_data.labels}
    test_losses.append(sess.run([avg_accuracy], feed_dict=feed_dict))

    print epoch, train_losses[-1], val_losses[-1], test_losses[-1]

0 [0.16979167] [0.16666669] [0.13333334]
1 [0.19270834] [0.16666669] [0.15000001]
2 [0.48541668] [0.41666669] [0.3666667]
3 [0.40625006] [0.35000002] [0.27777779]
4 [0.6052084] [0.55000007] [0.47222227]
5 [0.671875] [0.56666672] [0.56666672]
6 [0.73125005] [0.73333335] [0.65555561]
7 [0.81875008] [0.75000006] [0.70555568]
8 [0.88854176] [0.78333342] [0.77777779]
9 [0.90104169] [0.83333337] [0.80000001]
10 [0.92916673] [0.83333337] [0.85000002]
11 [0.93020833] [0.81666672] [0.80555564]
12 [0.95208323] [0.85000002] [0.86666673]
13 [0.9666667] [0.91666669] [0.92222226]
14 [0.97395837] [0.90000004] [0.9000001]
15 [0.984375] [0.93333334] [0.92222226]
16 [0.98541665] [0.93333334] [0.92777777]
17 [0.98958337] [0.9333334] [0.92777777]
18 [0.99166673] [0.95000005] [0.92222232]
19 [0.99270844] [0.95000005] [0.9333334]
20 [0.9937501] [0.9333334] [0.93888891]
21 [0.9937501] [0.95000005] [0.95000005]
22 [0.9958334] [0.95000005] [0.94999999]
23 [0.99479175] [0.9333334] [0.95555556]
24 [0.9979167] [0

In [186]:
# feed_dict = {diffs_pl:test_data.diff_maps, labels_pl:test_data.labels}
# pred = sess.run(prediction, feed_dict=feed_dict)
# missed = np.where(test_data.labels != pred)
# pred[missed]
# test_data.labels[missed]
# test_data.ids[missed]